# Project: Wrangling and Analyze Data

In [1]:
# Install the twitter API
!pip install tweepy

In [2]:
# Import Libraries
import os
import re
import time
import json
import tweepy
import requests
import webbrowser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlretrieve

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
if not os.path.exists('twitter_archive_enhanced.csv'): #confirm if file not downloaded already
    try:
        url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'
        urlretrieve(url, 'twitter_archive_enhanced.csv')
        print('File saved successfully')
    except OSError as e:
        print(e)

In [ ]:
df_archive = pd.read_csv('twitter_archive_enhanced.csv')
df_archive.head()

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
if not os.path.exists("image_predictions.tsv"):
    try:
        url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
        page = requests.get(url)
        print('Connection sucessful')
        with open('image_predictions.tsv', 'wb') as file:
            file.write(page.content)
            print('File saved successfully')
    except OSError as e:
        print(e)

In [ ]:
df_image = pd.read_csv('image_predictions.tsv', sep = '\t')
df_image.head()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
tweet_ids = list(df_archive.tweet_id)

file_name = 'tweet_ids.txt'
with open(file_name, 'w') as f:
  for id in tweet_ids:
    f.write(f'{str(id)}\n')

'''
Test if it can be printed line by line.

 with open(file_name, 'r') as f:
  tweet_ids = f.readline()
  print(tweet_ids)
'''

In [ ]:
Download tweets with API

In [ ]:
print(r'Error: Kindly provide the consumer_key, consumer_screet, access_token, '
      'access_screet and bearer_token')

#import tweepy
#from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = "Your Key Here"
consumer_secret = "Your Key Here"
access_token = "Your Key Here"
access_secret = "Your Key Here"
bearer_token = "Your Key Here"

#auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# Same result as the cell above but data not saved here
tweet_ids = df_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [ ]:
Read the tweet_json.txt file

In [ ]:
tweets = []
for line in open('tweet_json.txt', 'r'):
    tweets.append(json.loads(line))

In [ ]:
# Select columns to be used for the analysis
id = []
favorite_count = []
retweet_count = []
created_at = []
for value in tweets:
  id.append(value['id'])
  favorite_count.append(value['favorite_count'])
  retweet_count.append(value['retweet_count'])
  date = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(value['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
  created_at.append(date)


dict = {
    'Created_at' : created_at,
    'tweet_id' : id,
    'favorite_count' : favorite_count,
    'retweet_count' : retweet_count,
    }

df_tweet = pd.DataFrame(dict)

In [ ]:
df_tweet.head()

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [ ]:
df_archive
df_archive.head()

In [ ]:
df_image
df_image.head()

In [ ]:
df_tweet
df_tweet.head()

In [ ]:
df_archive.info()

In [ ]:
df_image.info()

In [ ]:
df_tweet.info()

In [ ]:
all_columns = pd.Series(list(df_archive) + list(df_image) + list(df_tweet))
all_columns[all_columns.duplicated()]

In [ ]:
df_archive.isnull().sum().sum()

In [ ]:
df_image.isnull().sum().sum()

In [ ]:
df_tweet.isnull().sum().sum()

In [ ]:
df_archive.describe()

In [ ]:
df_image.describe()

In [ ]:
df_tweet.describe()

In [ ]:
df_archive['in_reply_to_status_id'].unique()

In [ ]:
df_archive['in_reply_to_user_id'].unique()

In [ ]:
df_archive.sample(5)

In [ ]:
df_archive.tweet_id.duplicated().sum()

In [ ]:
df_archive.doggo.duplicated().sum()

In [ ]:
df_archive.doggo.value_counts()

In [ ]:
df_archive.pupper.value_counts()

In [ ]:
df_archive.source[0]

In [ ]:
df_image.sample(5)

In [ ]:
df_image.p1_dog.unique()

In [ ]:
df_image.p2_dog.unique()

In [ ]:
df_image.p3_dog.unique()

In [ ]:
df_tweet.sample(5)

### Quality issues
1. Archive table
    a. Change the datatype for some of the columns i.e. timestamp
    b. A lot of missing data in the features
    c. Missing values represented as "None"
    d. Expanded_url containing more than one url
    
2. Image table
    a. Lowercase for P1, P2, and P3 occassionally
    b. Text column not properly formatted

3. Tweet table
    a. Extract the date from Created_at column
    b. Rename the Created_at column as Timestamp to bridge uniformity

### Tidiness issues
1. P1, P2, and P3 should be formatted properly in the Image table

2. Remove html tags from the source column in the Archive table

3. Tweet_id in the Archive table is duplicated in the Image and Tweet tables

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data
archive = df_archive.copy()
image = df_image.copy()
tweet = df_tweet.copy()

### Issue #1: Missing Data

#### Define: archive: in_reply_to_status, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp contain little or no meaningful data and should be dropped

#### Code

In [ ]:
archive.drop(columns = ['in_reply_to_status_id', 'in_reply_to_user_id','retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp'], inplace = True)

#### Test

In [ ]:
archive.head()

### Issue #2: Data Type

1. archive: Timestamp is a datetime and not an object
2. archive: Tweet_id is an object not an integer
3. image: P2_dog is a boolean not an integer
4. tweet: Created_at is a datetime and not an integer

#### Define
Change the timestamp datatype to datetime, P2_dog to boolean, and tweet_id to object

#### Code

In [ ]:
archive['timestamp'] = pd.to_datetime(archive['timestamp'])
archive['tweet_id'] = archive['tweet_id'].astype(str)
image['tweet_id'] = image['tweet_id'].astype(str)
image['p2_dog'] =  image['p2_dog'].astype(bool)
tweet['Created_at'] = pd.to_datetime(tweet['Created_at'])
tweet['tweet_id'] = tweet['tweet_id'].astype(str)

#### Test

In [ ]:
archive.dtypes

In [ ]:
image.dtypes

In [ ]:
tweet.dtypes

### Issue #3: Format Column Names

#### Define
Change the created_at to timestamp in tweet table

#### Code

In [ ]:
archive.columns

In [ ]:
image.columns

In [ ]:
tweet.rename(columns= {"Created_at": "timestamp"}, inplace = True)

#### Test

In [ ]:
tweet.columns

### Issue #4: Format P1, P2, and P3 in the image table

#### Define
Replace "-" with space (" ") for P1, P2, and P3 in the image table

#### Code

In [ ]:
image.p1 = image.p1.str.replace("_", ",")
image.p2 = image.p2.str.replace("_", ",")
image.p3 = image.p3.str.replace("_", ",")

#### Test

In [ ]:
image.head()

### Issue #5: Format the timestamp and source in archive

#### Define
1. Remove html tags from the source column in the archive table
2. Make the timestamp to contain year, month, and day only
3. Choose only the expanded_url url that follows the normal pattern

#### Code

In [ ]:
archive['source']  = archive.source.str.extract('(http://\w+\.com/\w+/iphone)', expand = True)
archive['expanded_urls']  = archive.expanded_urls.str.extract('(https://\w+\.com/\w+/\w+/\d+/\w+/\d)', expand = True)

archive['timestamp'] = archive['timestamp'].astype(str)
tweet['timestamp'] = tweet['timestamp'].astype(str)
archive['timestamp'] = archive['timestamp'].str.extract('(\d{4}[-]\d{2}[-]\d{2})', expand = True)
tweet['timestamp'] = tweet['timestamp'].str.extract('(\d{4}[-]\d{2}[-]\d{2})', expand = True)
archive['timestamp'] = pd.to_datetime(archive['timestamp'])
tweet['timestamp'] = pd.to_datetime(tweet['timestamp'])

#### Test

In [ ]:
archive.head()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
twitter_archive_master = pd.merge(pd.merge(archive, image, on = "tweet_id"), tweet, on = "tweet_id")

In [ ]:
twitter_archive_master.head()

### It can be observed that both timestamp_x and timestamp_y are the same
1. Drop timestamp_y
2. Remove x from timestamp_x

In [ ]:
twitter_archive_master.drop(columns = ['timestamp_y'], inplace= True)
twitter_archive_master.rename(columns = {'timestamp_x':'timestamp'}, inplace = True)
twitter_archive_master.head()

### Data Engineering

In [ ]:
twitter_archive_master['month'] = twitter_archive_master.timestamp.dt.month_name()
twitter_archive_master['day'] = twitter_archive_master.timestamp.dt.day_name()

In [ ]:
twitter_archive_master.to_csv("twitter_archive_master.csv", index = False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Questions

1. Top ten months with the highest favorite_count
2. Top ten months with the hightest tweet_count
3. Influence of the day on the favorite_count
4. Finding the correlation between the Favorite and Retweet counts


# Insights

### Write function for the visualization

In [ ]:
df = twitter_archive_master.copy()

In [ ]:
def barhplot(x, y, xlabel, title):
    plt.figure(figsize=  (14, 8))
    plt.barh(x, y, align = 'center')
    plt.gca().invert_yaxis()
    plt.xlabel(xlabel, fontsize = 18)
    plt.title(title, fontsize = 18)
    plt.show();

### Top ten months with the highest favorite_count

In [ ]:
top = df.sort_values(by = 'favorite_count', ascending = False)
top_10 = top[['tweet_id', 'source', 'favorite_count', 'month']].head(10)
top_10

In [ ]:
barhplot(top_10.month, top_10.favorite_count, "Favorite Count", "Months with the highest favorite count")

### Top ten months with the highest retweet_count

In [ ]:
top = df.sort_values(by = 'retweet_count', ascending = False)
top_10 = top[['tweet_id', 'source', 'text', 'retweet_count', 'month']].head(10)
top_10.head()

In [ ]:
barhplot(top_10.month, top_10.retweet_count, "Retweet Count", "Months with the highest tweet count")

### Influence of the day on the favorite_count

In [ ]:
top = df.groupby(by = 'day')
top = df.sort_values(by = 'favorite_count', ascending = False)
top_10 = top[['tweet_id', 'source', 'text', 'favorite_count', 'day']].head(10)
top_10.head()

In [ ]:
barhplot(top_10.day, top_10.favorite_count, "Favorite Count", "Influence of the day on the favourite count")

### Finding the correlation between the Favorite and Retweet counts

In [ ]:
df.favorite_count.corr(df.retweet_count)

### Visualization

In [ ]:
plt.figure(figsize=(15, 13))
ax = plt.axes()
ax.scatter(df.favorite_count, df.retweet_count)

ax.set_xlabel('Favorite Count')
ax.set_ylabel('Retweet Count')
ax.set_title('Correlatiion Between the Favorite and Retweet Counts')

ax.axis('tight')

plt.show()

### Checking for other correlations

1. Between favorite_count and rating_numerator
2. Between favorite_count and rating_denominator
3. Between retweet_count and rating_numerator
4. Between retweet_count and rating_denominator

In [ ]:
df.favorite_count.corr(df.rating_numerator)

In [ ]:
df.favorite_count.corr(df.rating_denominator)

In [ ]:
df.retweet_count.corr(df.rating_numerator)

In [ ]:
df.retweet_count.corr(df.rating_denominator)